<h2><b>Introduction</b></h2><br>
<h3>The objective of this project is to create my personal database  on colleges offering Under Graduate course on Computer Application. Since I am thinking of taking upon that course I asked why not make my search for college easier by web scraping valuable information at once  rather than going over the site and spending hours clicking on the hyperlinks.<h3>

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.shiksha.com/it-software/colleges/bca-colleges-kolkata?rf=searchWidget&landing=ctp'
response = requests.get(url)

In [3]:
#cheching if the GET query was successful
response

<Response [200]>

In [4]:
#creating web scraping object
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
college_soup = soup.find_all('strong', {'class': 'instNamev2'})
addr_soup = soup.find_all('span', {'class': 'instLoc'})

In [8]:
#size of our datas
len(college_soup), len(addr_soup)

(30, 30)

<h3>Extracting the values for fees and course rating was kind of tricky since the tags had multiple variables under it. So I had to come up with a slightly different approach to extract valuable information from it.</h3>

In [13]:
uniqueLabl1_soup = soup.find_all('label', {'class': 'blockLabel'})
uniqueLabl2_soup = soup.find_all('div', {'class': 'valueTxt'})

In [15]:
#size of the above labls
len(uniqueLabl1_soup), len(uniqueLabl2_soup)

(180, 180)

<h3>As you can see the size of college_soup and addr_soup is 30 whereas the size of uniqeLabl1_soup and uniqueLabl2_soup is 180. A clear proof that the latter holds much more information than the other two. Also these values are not in generic form as you can see below. Its a bunch of html tags and parameters.</h3>

In [16]:
#as for now our data values looks like this
college_soup[:4]

[<strong class="instNamev2" title="MAKAUT - Maulana Abul Kalam Azad University of Technology">MAKAUT - Maulana Abul Kalam Azad University of Technology</strong>,
 <strong class="instNamev2" title="Adamas University">Adamas University</strong>,
 <strong class="instNamev2" title="Dinabandhu Andrews Institute of Technology and Management">Dinabandhu Andrews Institute of Technology and Management</strong>,
 <strong class="instNamev2" title="NSHM Knowledge Campus, Kolkata">NSHM Knowledge Campus, Kolkata</strong>]

In [17]:
addr_soup[:4]

[<span class="instLoc"><i class="ctpSprite ctpv2-loc"></i>Salt Lake City, Kolkata</span>,
 <span class="instLoc"><i class="ctpSprite ctpv2-loc"></i>Barasat, Kolkata</span>,
 <span class="instLoc"><i class="ctpSprite ctpv2-loc"></i>Patuli, Kolkata</span>,
 <span class="instLoc"><i class="ctpSprite ctpv2-loc"></i>Tollygunge, Kolkata</span>]

In [18]:
uniqueLabl1_soup[:4]

[<label class="blockLabel"><span class="truncateTxt">BCA</span> Courses</label>,
 <label class="blockLabel truncateTxt">Exams</label>,
 <label class="blockLabel">Total Fees</label>,
 <label class="blockLabel">Median Salary</label>]

In [19]:
uniqueLabl2_soup[:4]

[<div class="valueTxt"><a class="ripple dark" history="[object Object]" href="/university/makaut-maulana-abul-kalam-azad-university-of-technology-kolkata-51181/courses?s[]=8&amp;bc[]=33&amp;ed[]=et_20&amp;ct[]=130" location="[object Object]" match="[object Object]" staticcontext="[object Object]">1 Course</a></div>,
 <div class="valueTxt"><div class="exams-flex">─</div></div>,
 <div class="valueTxt">─</div>,
 <div class="valueTxt">─</div>]

In [28]:
colleges = [college_soup[i].text for i in range(len(college_soup))]
addresses = [addr_soup[i].text for i in range(len(addr_soup))]
uniq1 = [uniqueLabl1_soup[i].text for i in range(len(uniqueLabl1_soup))]
uniq2 = [uniqueLabl2_soup[i].text for i in range(len(uniqueLabl2_soup))]

<h3>Now the data converted into generic form. </h3>

In [21]:
colleges[:4]

['MAKAUT - Maulana Abul Kalam Azad University of Technology',
 'Adamas University',
 'Dinabandhu Andrews Institute of Technology and Management',
 'NSHM Knowledge Campus, Kolkata']

In [22]:
addresses[:4]

['Salt Lake City, Kolkata',
 'Barasat, Kolkata',
 'Patuli, Kolkata',
 'Tollygunge, Kolkata']

In [29]:
uniq1[:4]

['BCA Courses', 'Exams', 'Total Fees', 'Median Salary']

In [30]:
uniq2[:4]

['1 Course', '─', '─', '─']

In [31]:
#creating a dataset
data_to_put = {
    'college' : colleges,
    'addresses' : addresses,
}

In [33]:
df = pd.DataFrame(data_to_put)
df.head()

,college,addresses
0,MAKAUT - Maulana Abul Kalam Azad University of...,"Salt Lake City, Kolkata"
1,Adamas University,"Barasat, Kolkata"
2,Dinabandhu Andrews Institute of Technology and...,"Patuli, Kolkata"
3,"NSHM Knowledge Campus, Kolkata","Tollygunge, Kolkata"
4,Brainware University,"Barasat, Kolkata"


<h3>As stated before the collge and address values were fine from the start but the unique values had to be manipulated a little. So to solve the problem of too many values, I converted the unique labels to a temporary dataframe whereby I extracted the values for 'Total Fees' label and 'Course Rating' Label only.</h3>

In [35]:
temp_data = {
    'labels' : uniq1,
    'values' : uniq2,
}

temp = pd.DataFrame(temp_data)
temp.head()

,labels,values
0,BCA Courses,1 Course
1,Exams,─
2,Total Fees,─
3,Median Salary,─
4,Course Rating,5.01


In [36]:
temp.set_index('labels', inplace=True)

In [37]:
temp.head()

,values
labels,
BCA Courses,1 Course
Exams,─
Total Fees,─
Median Salary,─
Course Rating,5.01


In [38]:
total_fees = temp.loc['Total Fees']

In [41]:
len(total_fees)

30

In [42]:
ratings = temp.loc['Course Rating']

In [43]:
len(ratings)

30

In [47]:
fees_array = np.array(total_fees['values']).reshape(30, 1)
ratings = np.array(ratings['values']).reshape(30, 1)

In [48]:
df['total_fees'] = fees_array
df['ratings'] = ratings

In [51]:
df

,college,addresses,total_fees,ratings
0,MAKAUT - Maulana Abul Kalam Azad University of...,"Salt Lake City, Kolkata",─,5.01
1,Adamas University,"Barasat, Kolkata",₹ 3.26 L,─
2,Dinabandhu Andrews Institute of Technology and...,"Patuli, Kolkata",₹ 2.13 L,4.523
3,"NSHM Knowledge Campus, Kolkata","Tollygunge, Kolkata",₹ 3.3 L,4.113
4,Brainware University,"Barasat, Kolkata",₹ 2.8 L,4.17
5,Eminent College of Management and Technology -...,"Barasat, Kolkata",₹ 1.25 L,4.82
6,"Techno India University, Kolkata","Salt Lake City, Kolkata",─,4.58
7,IEM Kolkata - Institute of Engineering and Man...,"Salt Lake City, Kolkata",─,4.567
8,"Amity University, Kolkata","Kadampukur, Kolkata",₹ 3.24 L,4.22
9,Techno International New Town,"Rajarhat, Kolkata",─,3.77


<h3> And this is our data in its all full glory. </h3>